In [1]:
# import metrics
import os
import math
import random
import time
from PIL import Image

import torch # pytorch의 tensor와 그와 관련된 기본 연산 등을 지원
import torch.nn as nn # 여러 딥러닝 layer와 loss, 함수 등을 클래스 형태로 지원
import torch.nn.functional as F # 여러 loss, 함수 등을 function 형태로 지원
import torch.optim as optim # 여러 optimizer를 지원
import torchvision.models as models

dev = 'cuda' if torch.cuda.is_available() else 'cpu'

### CosFace

![Architecture](img/Cosface.PNG)

ArcFace와 비슷한 모델 중 하나로 CosFace라는 모델이 있습니다. 위 Loss function은 CosFace의 loss function에 해당합니다. 위 loss function을 참고하여 아래의 ??? 부분을 채워주세요.

In [2]:
class CosMarginProduct(nn.Module):
    '''
    목적 : Cosface 의 last fc layer의 구현
    
    인자 :
    in_features : feature의 dimension
    out_features : class 개수
    '''
    def __init__(self, in_features, out_features, s=30.0, m=0.1):
        super(CosMarginProduct, self).__init__()        
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        
        # fc의 parameter 만들기 : (in_features x out_features)의 크기를 갖는 FloatTensor로 만들 것
        self.weight = nn.Parameter(torch.zeros([out_features, in_features], dtype=torch.float32)).to(dev)
        nn.init.xavier_uniform_(self.weight)

    def forward(self, input, label):
        '''
        Step 1. cos(theta)-m 계산하기
        '''

        # cos_theta = (x / ||x||) * (w * ||w||) 를 이용해 cosine_theta 구하기
        # 어느 dimension을 기준으로 normalization을 해서 torch.mm에 넘겨줘야 할까요?
        cos = torch.mm(F.normalize(input, dim=1), F.normalize(self.weight, dim=1))
        
        # cos_theta - m 구하기
        cos_m = cos - self.m
        
        '''
        Step 2. cos(theta)-m 에서 dim=1에 기준으로 y_i에 해당하는 부분만 남기고 나머지는 cos(theta)로 되돌리기 
        '''
        one_hot = torch.zeros(cos.size()).to(dev)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        output = (one_hot * cos_m) + (torch.abs(one_hot - 1) * cos)
        
        '''
        Step 3. 최종 output 계산하기
        '''
        output *= self.s
        
        return output

### SphereFace

![Architecture](img/Sphereface.PNG)

ArcFace와 비슷한 모델 중 하나로 SphereFace 모델이 있습니다. 위 Loss function은 SphereFace의 loss function에 해당합니다. 위 loss function을 참고하여 아래의 ??? 부분을 채워주세요.

In [19]:
class SphereMarginProduct(nn.Module):
    '''
    목적 : Sphereface의 last fc layer의 구현
    
    인자 :
    in_features : feature의 dimension
    out_features : class 개수
    '''
    def __init__(self, in_features, out_features, m=4):
        super(SphereMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.m = m
        
        # fc의 parameter 만들기 : (out_features x in_features)의 크기를 갖는 FloatTensor로 만들 것
        self.weight = nn.Parameter(torch.zeros([out_features, in_features], dtype=torch.float32)).to(dev)
        nn.init.xavier_uniform_(self.weight)

    def forward(self, input, label):
        '''
        Step 1. cos(m * theta) 계산하기
        '''

        # cos_theta = (x / ||x||) * (w * ||w||) 를 이용해 cosine_theta 구하기
        # 어느 dimension을 기준으로 normalization을 해서 F.linear에 넘겨줘야 할까요?
        cos = F.linear(F.normalize(input, dim=1), F.normalize(self.weight, dim=1))
        
        # cos(m * theta) 구하기. 논문에서 m=4로 제시하고 있으므로 m=4 일 경우에 대해서만 계산합니다.
        # 효율성을 위해 arccos 등의 다른 연산 없이 위에서 얻은 cos만을 사용해 계산합니다.
        cos_m = 8.0 * cos**4 - 8.0 * cos**2 + 1     # cos(2x) = 2*cos(x)**2 - 1
        
        '''
        Step 2. cos(m * theta) 에서 dim=1에 기준으로 y_i에 해당하는 부분만 남기고 나머지는 cos(theta)로 되돌리기 
        '''
        one_hot = torch.zeros(cos.size()).to(dev)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        output = (one_hot * cos_m) + (torch.abs(one_hot - 1) * cos)
        
        '''
        Step 3. 최종 output 계산하기
        '''
        x_norm = torch.norm(input)
        output *= x_norm
        return output
    

### Backbone network

ResNet-101을 이용하여 Backbone network를 구현합니다. 아래 코드의 ??? 부분을 채워주세요.

In [20]:
class FeatureNet_101(nn.Module):
    def __init__(self, dim_feature):
        super(FeatureNet_101, self).__init__()
        resnet = models.resnet101(pretrained=False)

        # resnet-101의 마지막 Conv layer block 까지 남기고 그 뒷부분은 잘라냅니다.
        # resnet-101 의 구조를 print해서 어디를 잘라야할 지 알 수 있습니다.
        self.backbone = nn.Sequential(* list(resnet.children())[:-1])
        
        # 마지막 Conv layer block 부분 이후로 붙는 layer들입니다.
        # resnet-101 의 구조를 print해서 어떻게 뒤 쪽 layer를 디자인 해야할 지 알 수 있습니다.
        self.bn_4 = nn.BatchNorm2d(2048, eps=1e-5, momentum=0.1, affine=True, track_running_stats=True)
        self.dropout = nn.Dropout()
        self.fc = nn.Linear(2048, dim_feature)
        self.bn_5 = nn.BatchNorm1d(dim_feature)
        
    def forward(self, x):
        out = self.backbone(x)
        out = self.bn_4(out)
        out = self.dropout(out)
        out = out.view(out.size(0), -1)

        out = self.fc(out)
        out = self.bn_5(out)
        return out

### FaceNet

위에서 구현한 각 모델의 마지막 FC layer들과 Backbone network를 합쳐서 하나의 얼굴인식모델을 만듭니다.
아래 코드의 ??? 부분을 채워주세요.

In [21]:
class FaceNet(nn.Module):
    '''
    ArcMarginProduct와 FeatureNet-50 을 결합한 ArcFace 모델의 구현
    '''
    def __init__(self, feature_dim, cls_num, model_type='Cosface'):
        super(FaceNet, self).__init__()
        self.feature_net = FeatureNet_101(feature_dim)
        
        if model_type == 'Cosface':
            self.classifier = CosMarginProduct(feature_dim, cls_num)
        elif model_type == 'Sphereface':
            self.classifier = SphereMarginProduct(feature_dim, cls_num)

    # 끝까지 Forward 하여 logit을 return
    def forward(self, x, label):
        out = self.classifier(x, label)
        return out
    
    # Feature를 extract
    def extract_feature(self, x):
        out = self.feature_net(x)
        return out

In [22]:
# 두 input 이미지의 유사도를 측정하는데 사용되는 cosine similarity

def cos_dist(x1, x2):
    return torch.sum(x1 * x2) / (torch.norm(x1) * torch.norm(x2))

### FaceNet

FaceNet을 이용하여 두 input 사이의 similarity를 계산합니다.

In [23]:
# 두 input입니다.
x_1 = torch.randn(1, 3, 128, 128).to(dev)
x_2 = torch.randn(1, 3, 128, 128).to(dev)

# 각 model을 만듭니다. 이 모델에서 사용하는 feature의 dim은 512고 class는 총 1000개가 있습니다.
SphereFaceNet = FaceNet(feature_dim=512, cls_num=1000, model_type='Sphereface').to(dev)
CosFaceNet = FaceNet(feature_dim=512, cls_num=1000, model_type='Cosface').to(dev)

# test를 위해 model을 test phase로 변경합니다.
# 특정 layer는 training과 test 떄 다르게 동작하므로 이 설정은 필수입니다. (ex. dropout, batchnorm ...) 
SphereFaceNet.eval()
CosFaceNet.eval()

# x_1, x_2로부터 SphereFace 모델을 이용해 feature를 추출합니다.
feature_1 = SphereFaceNet.extract_feature(x_1)
feature_2 = SphereFaceNet.extract_feature(x_2)

# 두 feature의 유사도를 계산합니다.
sim = cos_dist(feature_1, feature_2)
print('SphereFace에서 두 input의 유사도는 %f 입니다.' % sim.item())

# x_1, x_2로부터 CosFace 모델을 이용해 feature를 추출합니다.
feature_1 = CosFaceNet.extract_feature(x_1)
feature_2 = CosFaceNet.extract_feature(x_2)

# 두 feature의 유사도를 계산합니다.
sim = cos_dist(feature_1, feature_2)
print('CosFace에서 두 input의 유사도는 %f 입니다.' % sim.item())

SphereFace에서 두 input의 유사도는 0.999979 입니다.
CosFace에서 두 input의 유사도는 0.999982 입니다.


In [24]:
label_1 = torch.zeros([1, 1000]).to(dev)
label_2 = torch.zeros([1, 1000]).to(dev)
label_1[:, 0] = 1.0
label_2[:, 1] = 1.0
# x_1, x_2로부터 SphereFace 모델을 이용해 feature를 추출합니다.
feature_1 = SphereFaceNet.extract_feature(x_1)
feature_2 = SphereFaceNet.extract_feature(x_2)

In [25]:
SphereFaceNet(feature_1, label_1)

RuntimeError: invalid argument 4: Index tensor must have same size as output tensor apart from the specified dimension at C:/w/1/s/windows/pytorch/aten/src\THC/generic/THCTensorScatterGather.cu:324

In [8]:
import matplotlib.pyplot as plt
import torchvision

In [9]:
torchvision.utils.save_image(x_2, 'test.jpg')

In [10]:
torchvision.utils.save_image(x_1, 'test2.jpg')

In [11]:
feature_1.shape

torch.Size([1, 512])